In [ ]:
%load_ext autoreload
%autoreload 2

In [114]:
import os
import json
from pathlib import Path
from os.path import join
os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd
import pandas as pd
import numpy as np

from util.files import *
from util.const import *
from util.ddfs import *

In [115]:
# We're only using single county for v0.1
fips = FIPS[0]
# STATE ABBR and NATION will be derived from FIPS, one day
stateabbr = STATEABBR[0]
nation = NATION[0]

# Prepare data for ensemble generation

## Load and subset exposure data

In [116]:
# Load the single family homes,
# the fd_id/reference file
# the fd_id/depths file
# the fd_id flood zone file
nsi_struct = gpd.read_file(join(EXP_DIR_I, fips, 'nsi_sf.gpkg'))
nsi_ref = pd.read_parquet(join(EXP_DIR_I, fips, 'nsi_ref.pqt'))
nsi_depths = pd.read_parquet(join(EXP_DIR_I, fips, 'nsi_depths.pqt'))
nsi_fz = pd.read_parquet(join(EXP_DIR_I, fips, 'nsi_fz.pqt'))

In [117]:
# Filter to properties with > 0 
depths_df = nsi_depths[nsi_depths.iloc[:,1:].sum(axis=1) > 0].set_index('fd_id')
# Rename columns with "depth" key
depths_df.columns = ['depth_' + x for x in depths_df.columns]

In [118]:
# Need foundation type, number stories, structure value
# for our ensemble. Structure value will be the center of 
# the distribution and will be passed to the loss estimation
# function. Foundation type will be drawn from the implicit
# distribution in the NSI data. For each census block, 
# we are going to get the multinomial probabilities of 
# a building having a certain foundation type & number of stories
# Ideally, we would do this conditioned on prefirm but the
# building year column is based on median year built from ACS
# data
# From the foundation type that is drawn from the multinomial in 
# the ensemble, we will get the FFE from the distribution 
# defined in the code for the Wing et al. 2022 paper
# The point estimate version will just use default values

# Start by retaining only relevant columns in nsi_struct
# Then subset this and nsi_ref to the fd_id in nsi_depths
# We do need sqft for elevation cost or floodproof estimates

# Normally we would only keep the below, but I'm commenting those out
# because we also want to keep found_ht
# keep_cols = ['fd_id', 'occtype', 'val_struct']
keep_cols = ['fd_id', 'occtype', 'val_struct', 'bldgtype',
             'found_type', 'found_ht', 'sqft']
nsi_res = nsi_struct[keep_cols]

# Let's merge in refs into nsi_res
nsi_res = nsi_res.merge(nsi_ref, on='fd_id')

# We're also going to merge in fzs
nsi_res = nsi_res.merge(nsi_fz[['fd_id', 'fld_zone']], on='fd_id')

# Split occtype to get the number of stories and basement
# We only need to keep stories for the purposes
# of estimating the distribution that stories comes from
# We will draw basement from the foundation type
# distribution which also gives us first floor elevation
structs = nsi_res['occtype'].str.split('-').str[1]
basements = structs.str[2:]
stories = structs.str[:2]

nsi_res = nsi_res.assign(stories=stories)

# Retain only the rows that correspond to structures
# that are exposed to flood depths
nsi_res_f = nsi_res[nsi_res['fd_id'].isin(nsi_depths['fd_id'])].set_index('fd_id')

# Merge in the depths to the struct df you are working with
# Also merge in the refs - note that there are inconsistencies
# with the cbfips column from nsi directly and the
# block data downloaded from the census webpage
# You retain more structures if you use the block data we
# downloaded in UNSAFE
full_df = (nsi_res_f.join(depths_df)).reset_index()

# ^ This dataset can be directly used for estimating the 
# benchmark losses of using NSI as-is
# Use the Hazus DDFs with no uncertainty


In [119]:
# Let's get the fld_zone column processed for the way it needs
# to be done for using hazus ddfs
# Get the first character of the flood zone and only retain it
# if it's a V zone. We are going to use A zone for A and outside
# (if any) flood zone depth exposures
ve_zone = np.where(full_df['fld_zone'].str[0] == 'V',
                   'V',
                   'A')
full_df = full_df.assign(fz_ddf = ve_zone)

## Get parameters for structures

In [120]:
# We are  going to use nsi_struct merged with refs
# to determine the multinomial probabilities of basement
# and number stories (binomial) from ref level which matches 
# (to the best of our ability)
# up with NSI tech reference on where data is randomly assigned
# from. While there are maps from parcel data, where available, 
# it's not clear which entries have this non-random assignment. 
# In addition, it is known that parcel aggregation datasets like
# ZTRAX may have data errors. The sources the NSI used
# have unknown validation/accuracy so we can treat these as
# part of estimating the distribution to draw from

# The method for estimating number of stories is based on assignment
# from parcel data. Where missing, square footage is divided by the 
# structure's footprint (when sq. ft. is missing, they take 86% of
# the structure's footprint as sq. ft). If > 1.25,
# a second floor is assumed
# If no footprint is available, 
# stories is randomly assigned from a distribution that varies by
# year built and census region. So, we can use census ref again
# here

# The methodology for the structure valuation is obscure
# and there is no reporting on how accurate it is to some
# observed data on market values

# There are not nearly enough observations at the block level
# to reliably estimate the parameter for binomial # stories
# or multinomial foundation type. Sometimes just one observation
# in general. Tract appears to have enough
# This check is based on the subset of tracts (or other ref)
# in nsi_res that are also in full_df (these are the ones) we need
# the probabilities for
# TODO - STRUCT_REF should either be a config, 
# or something identified on-the-fly based on 
# whether we have enough observations at block or blockgroup
# before "rolling back" to coarser resolution to get
# proportions

STRUCT_REF = 'tract_id'
# Note that this approach assumes there is no correlation of
# the foundation type proportions and number of stories
# with features that vary within a census tract. Might not be
# the right thing to do. That's a TODO
struct_tot = nsi_res[nsi_res[STRUCT_REF].isin(full_df[STRUCT_REF])]

### Number of stories 

In [121]:
# Get the total number of structures w/ number of stories 
# in each block gruop
stories_sum = struct_tot.groupby([STRUCT_REF, 'stories']).size()
# Then get the proportion
stories_prop = stories_sum/struct_tot.groupby([STRUCT_REF]).size()
# Our parameters can be drawn from this table based on the bg_id
# of a structure we are estimating losses for
stories_param = stories_prop.reset_index().pivot(index=STRUCT_REF,
                                                 columns='stories',
                                                 values=0).fillna(0)
# Since it's a binomial distribution, we only need to specify
# one param. Arbitrarily choose 1S
# Round the param to the hundredth place
# Store in a dict
stories_param = stories_param['1S'].round(2)
STRY_DICT = dict(stories_param)

### Foundation types

In [122]:
# Repeat procedure above
found_sum = struct_tot.groupby([STRUCT_REF, 'found_type']).size()
found_prop = found_sum/struct_tot.groupby([STRUCT_REF]).size()
found_param = found_prop.reset_index().pivot(index=STRUCT_REF,
                                             columns='found_type',
                                             values=0).fillna(0)

# We want a dictionary of bg_id to a list of B, C, S
# for direct use in our multinomial distribution draw
# Store params in a list (each row is bg_id and corresponds to
# its own probabilities of each foundation type)
params = found_param.values.round(2)
# Then create our dictionary
FND_DICT = dict(zip(found_param.index, params))

## Load depth damage functions

In [123]:
# Load DDFs
naccs_ddfs = pd.read_parquet(join(VULN_DIR_I, 'physical', 'naccs_ddfs.pqt'))
hazus_ddfs = pd.read_parquet(join(VULN_DIR_I, 'physical', 'hazus_ddfs.pqt'))
hazus_nounc = pd.read_parquet(join(VULN_DIR_I, 'physical', 'hazus_ddfs_nounc.pqt'))

# Load helper dictionaries
with open(join(VULN_DIR_I, 'physical', 'hazus.json'), 'r') as fp:
    HAZUS_MAX_DICT = json.load(fp)

with open(join(VULN_DIR_I, 'physical', 'hazus_nounc.json'), 'r') as fp:
    HAZUS_MAX_NOUNC_DICT = json.load(fp)

with open(join(VULN_DIR_I, 'physical', 'naccs.json'), 'r') as fp:
    NACCS_MAX_DICT = json.load(fp)

# Generate Ensemble

In [124]:
# We need a randon number generator
rng = np.random.default_rng()

In [125]:
# Need to create a dataframe w/ N_SOW rows for each fd_id
# From full_df, keep fd_id, val_struct, bg_id, and the
# depth columns. 
# df.loc[np.repeat(df.index, N)].reset_index(drop=True)
# With this approach, we can do everything in a vectorized
# form by passing array_like data of size N*len(df)
# to different rng() calls to get all the draws from
# distributions that we need

# We drop these for the ensemble because we don't need them
drop_cols = ['occtype', 'found_type', 'block_id', 'fld_zone',
             'stories']

ens_df = full_df.drop(columns=drop_cols)
ens_df = ens_df.loc[np.repeat(ens_df.index, N_SOW)].reset_index(drop=True)
print('Created Index for Ensemble')

Created Index for Ensemble


## Sample structure characteristics

In [126]:
# Values
# Draw from the structure value distribution for each property
# normal(val_struct, val_struct*CF_DET) where these are array_like
# I also want to treat this as truncated
# on the lower end since there is a risk of drawing impossibly
# low numbers (like negative) with this approach
# https://github.com/kieranrcampbell/blog-notebooks/blob/master/
# Fast%20vectorized%20sampling%20from%20truncated%
# 20normal%20distributions%20in%20python.ipynb
# outlines an approach to use numpy to do a truncated sample
# TODO move this to a util file
def truncnorm_rvs_recursive(x, sigma, lower_clip):
    rng = np.random.default_rng()
    q = rng.normal(x, sigma)
    if np.any(q < lower_clip):
        # Adjustment to the code provided to index the sigma vector
        q[q < lower_clip] = truncnorm_rvs_recursive(x[q < lower_clip],
                                                    sigma[q < lower_clip],
                                                    lower_clip)

    return q
# Using 20000 as an artificial, arbitrary lower bound on value
values = truncnorm_rvs_recursive(ens_df['val_struct'],
                                 ens_df['val_struct']*COEF_VARIATION,
                                 20000)

print('Draw values')

# Draw from the #stories distribution
# We do this by mapping ens_df values with STRY_DICT
# and passing this parameter to rng.binomial()
# We also need to create an array of 1s with length
# N_SOW * len(full_df) - i.e. len(ens_df)
# full_df['bg_id'].map(STRY_DICT)
bin_n = np.ones(len(ens_df), dtype=np.int8)
bin_p = ens_df[STRUCT_REF].map(STRY_DICT).values
# This gives us an array of 0s and 1s
# Based on how STRY_DICT is defined, the probability of
# success parameter corresponds to 1S, so we need to
# swap out 1 with 1S and 0 with 2S
stories = rng.binomial(bin_n, bin_p)
stories = np.where(stories == 1,
                   '1S',
                   '2S')

print('Draw stories')

# Draw from the fnd_type distribution
# We do the same thing as above but with
# the FND_DICT. This is a multinomial distribution
# and 0, 1, 2 correspond to B, C, S
# We get an array returned of the form 
# [0, 0, 1] (if we have Slab foundation, for example)
# so we need to transform this into the corresponding
# foundation type array
# Can do this with fnds[fnds[0] == 1] = 'B'
# fnds[fnds[1]] == 1] = 'C' & fnds[fnds[2] == 1] = 'S'
# One way to do the mapping is by treating each
# row-array as a binary string and converting it
# to an int
# So you get [a, b, c] => a*2^2 + b*2^1 + c*2^0
# This uniquely maps to 4, 2, and 1
# So we can create a dict for 4: 'B', 2: 'C', and 1: 'S'
# and make it a pd.Series() (I think this is useful because
# pandas can combine this with the 1S and 2S string easily
# into a series and we'll need to use that bld_type
# for the other dicts we have)

# This is our ens_df index aligned multinomial
# probabilities array
# np.stack makes sure the dtype is correct
# Not sure why it is cast to object dtype if
# I call .values, but this works...

mult_p = np.stack(ens_df[STRUCT_REF].map(FND_DICT))
# This is our map of binary string/int
# conversions to the foundation type
bin_str_map = {4: 'B', 2: 'C', 1: 'S'}
# We need our np.ones array 
mult_n = np.ones(len(ens_df), dtype=np.int8)
# Draw from mult_p
fnds = rng.multinomial(mult_n, mult_p)
# Create a series of 4, 2, and 1 from the binary strings
# This code accomplishes the conversion outlined in the
# note above and comes from this stackoverflow post
# https://stackoverflow.com/questions/41069825/
# convert-binary-01-numpy-to-integer-or-binary-string
fnds_ints = pd.Series(fnds.dot(2**np.arange(fnds.shape[1])[::-1]))
# Replace these values with the fnd_type
fnd_types = fnds_ints.map(bin_str_map)

print('Draw foundation type')

# We take fnd_types for two tasks now
# First, if B, it's WB type home and we
# combine this with stories to get the bld_type
# This is naccs_ddf_type 
# We combine bld_type with fz_ddf to get hazus_ddf_type
# For our case study, it turns out we will use the same hazus
# ddf for the basement houses (_A) since no V zone houses
# For no basement, hazus_ddf_type does not add the _fz

# Let's get bld_type
# Basement type from fnd_types
base_types = np.where(fnd_types == 'B',
                      'WB',
                      'NB')

# Combine 1S and this
bld_types = pd.Series(stories) + pd.Series(base_types)

# In theory, bld_type is naccs_ddf_type. No need to 
# take this storage up in practice... just refer to bld_type
# when needed
# For WB homes, hazus_ddf_type is bld_types + '_' + ens_df['fz_ddf']
# For NB homes, it's bld_types
# It makes practical sense to create a new series for this
hazus_ddf_types = np.where(base_types == 'WB',
                           bld_types + '_' + ens_df['fz_ddf'],
                           bld_types)


# We are going to use the fnd_type to draw from the
# FFE distribution
# Need to use np.stack to get the array of floats
tri_params = np.stack(fnd_types.map(FFE_DICT))

# Can use [:] to access like a matrix and directly input to 
# rng.triangular
# 0, 1, and 2 are column indices corresponding to left,
# mode, and right
# We round this to the nearest foot
ffes = np.round(rng.triangular(tri_params[:,0],
                               tri_params[:,1],
                               tri_params[:,2]))

print('Generated Structure Characteristics')


Draw values
Draw stories
Draw foundation type
Generated Structure Characteristics


## Estimate losses

In [127]:
# First, we adjust depths by FFE
# And will store this in a df
depth_cols = [x for x in full_df.columns if 'depth' in x]
depth_ffe_df = ens_df[depth_cols].subtract(ffes, axis=0).round(1) 

In [128]:
# Now, we are going to loop through each return period
# and estimate losses for NACCS and HAZUS using our helper
# functions for each of these

for d_col in depth_ffe_df.columns:
    rp = d_col.split('_')[-1]
    naccs_loss[rp] = est_naccs_loss(pd.Series(bld_types),
                                    depth_ffe_df[d_col],
                                    naccs_ddfs,
                                    NACCS_MAX_DICT)
    hazus_loss[rp] = est_hazus_loss(pd.Series(hazus_ddf_types),
                                    depth_ffe_df[d_col],
                                    hazus_ddfs,
                                    HAZUS_MAX_DICT)

    print('Estimate Losses for NACCS & Hazus, RP: ' + rp)

# Then, we convert these to dataframes
hazus_df = pd.DataFrame.from_dict(hazus_loss)
naccs_df = pd.DataFrame.from_dict(naccs_loss)

# And we use a binomial rv to determine if we use the hazus or naccs
# loss estimate for a particular SOW (across RPs right now)
# You would do this within the loop above if you wanted
# to change the damage function across RPs in a particular SOW
# I think it makes more sense to say that in a particular SOW,
# the flood-frequency relationship has a particular form
# (like, in a bootstrapped sense), and the way this is
# implemented now is consistent with that
# Binomial
random_loss = rng.binomial(1, .5, size=len(ens_df))

# Get indices to take from each df
hazus_ind = (random_loss == 0)
naccs_ind = (random_loss == 1)

# Concat subsetted dataframes
losses_df = pd.concat([hazus_df.loc[hazus_ind],
                       naccs_df.loc[naccs_ind]], axis=0).sort_index()
# Rename columns to make it more clear what this is
losses_df.columns = ['rel_dam_' + x for x in losses_df.columns]

# Add a column indicating NACCS or Hazus ddf
losses_df.loc[hazus_ind, 'ddf'] = 'HAZUS'
losses_df.loc[naccs_ind, 'ddf'] = 'NACCS'

print('Randomly assigned NACCS or HAZUS Loss')

# Add clearer column names to the ffe dataframe
depth_ffe_df.columns = ['ffe_' + x for x in depth_ffe_df.columns]

# Concat for our full ensemble
ens_df = pd.concat([ens_df, losses_df, depth_ffe_df,
                    pd.Series(stories, name='stories'),
                    pd.Series(fnd_types, name='fnd_type'),
                    pd.Series(ffes, name='ffe'),
                    pd.Series(values, name='val_s')],
                   axis=1)


# Get relative damage columns
rel_cols = [x for x in ens_df.columns if 'rel_dam' in x]
# For each relative damage column, scale by val_s, the structure
# value realization
for col in rel_cols:
    rp = col.split('_')[-1]
    ens_df['loss_' + rp] = ens_df[col].values*ens_df['val_s'].values

print('Obtained Full Ensemble')


Estimate Losses for NACCS & Hazus, RP: 500
Estimate Losses for NACCS & Hazus, RP: 100
Estimate Losses for NACCS & Hazus, RP: 50
Estimate Losses for NACCS & Hazus, RP: 10
Randomly assigned NACCS or HAZUS Loss
Obtained Full Ensemble


In [129]:
# Now we calculate EAL
# We will use trapezoidal approximation for this
# Using trapezoid method and adding bin of lowest probability
# events to obtain expected annual 

# We want a list of return periods sorted from most frequent
# to least frequent
# We can do this by removing 'depth_' from our depth_ffe_df columns
# and then sorting that list in ascending order
# From this, we can define a probability equivalent list
# and then a list indexing loss columns as well
rp_list = sorted([int(x.split('_')[-1]) for x in depth_ffe_df.columns])
p_rp_list = [round(1/int(x), 4) for x in rp_list]
loss_list = ['loss_' + str(x) for x in rp_list]

# Then we create an empty series
eal = pd.Series(index=ens_df.index).fillna(0)

# We loop through our loss list and apply the 
# trapezoidal approximation
# We need these to be sorted from most frequent
# to least frequent
for i in range(len(loss_list) - 1):
    loss1 = ens_df[loss_list[i]]
    loss2 = ens_df[loss_list[i+1]]
    rp1 = p_rp_list[i]
    rp2 = p_rp_list[i+1]
    eal += (loss1 + loss2)*(rp1-rp2)/2
final = eal + ens_df[loss_list[-1]]*p_rp_list[-1]

# This is the final trapezoid to add in
final_eal = eal + ens_df[loss_list[-1]]*p_rp_list[-1]
print('Calculated EAL')
# Add eal columns to our dataframe
ens_df = pd.concat([ens_df, pd.Series(final_eal, name='eal')],
                   axis=1)

# Let's also get the SOW index - start at 0
sow_ind = np.arange(len(ens_df))%N_SOW
ens_df = pd.concat([ens_df, pd.Series(sow_ind, name='sow_ind')], axis=1)

print('Generated loss ensemble')

Calculated EAL
Generated loss ensemble


In [132]:
# Write out our ensemble dfs
ens_out_filep = join(FO, 'ensemble.pqt')
prepare_saving(ens_out_filep)
ens_df.to_parquet(ens_out_filep)

# Generate losses without uncertainty

In [131]:
# We are taking the full_df dataframe values as-is
# to get eal estimates for our benchmark
# From occtype, get 1SNB, etc. and combine with fz_ddf
# for our ddf_id column
b_types = full_df['occtype'].str.split('-').str[1]

hazus_ddf_types = pd.Series(np.where(b_types.str[-2:] == 'WB',
                                     b_types + '_' + full_df['fz_ddf'],
                                     b_types))

# We need to get the depth columns translated into ffe adjusted
# exposure depths
# We can subset on depths_df to get a depth dataframe
# and then we can subtract it by the found_ht column
# for our adjustment
# Merge to get indices aligned
depth_cols = [x for x in full_df.columns if 'depth' in x]
depth_ffe_df = full_df[depth_cols].subtract(full_df['found_ht'], axis=0).round(1)

# Loop through return periods for losses
hazus_def_loss = pd.DataFrame()
for d_col in depth_ffe_df.columns:
    rp = d_col.split('_')[-1]
    hazus_def_loss[rp] = est_hazus_loss_nounc(hazus_ddf_types,
                                              depth_ffe_df[d_col],
                                              hazus_nounc,
                                              HAZUS_MAX_NOUNC_DICT)

    print('Estimate Losses for Hazus Default, RP: ' + rp)

# Same processing as before for column names
hazus_def_loss.columns = ['rel_dam_' + x for x in hazus_def_loss.columns]
# Adjust depth_ffe_df columns
depth_ffe_df.columns = ['ffe_' + x for x in depth_ffe_df.columns]

# Join dataframes
hazus_def = pd.concat([full_df, hazus_def_loss, depth_ffe_df],
                       axis=1)

# Get losses from rel_dam columns
for col in hazus_def_loss.columns:
    rp = col.split('_')[-1]
    hazus_def['loss_' + rp] = hazus_def[col]*hazus_def['val_struct']

# We want a list of return periods sorted from most frequent
# to least frequent
# We can do this by removing 'depth_' from our depth_ffe_df columns
# and then sorting that list in ascending order
# From this, we can define a probability equivalent list
# and then a list indexing loss columns as well
rp_list = sorted([int(x.split('_')[-1]) for x in depth_ffe_df.columns])
p_rp_list = [round(1/int(x), 4) for x in rp_list]
loss_list = ['loss_' + str(x) for x in rp_list]

# Then we create an empty series
eal = pd.Series(index=full_df.index).fillna(0)

# We loop through our loss list and apply the 
# trapezoidal approximation
for i in range(len(loss_list) - 1):
    loss1 = hazus_def[loss_list[i]]
    loss2 = hazus_def[loss_list[i+1]]
    rp1 = p_rp_list[i]
    rp2 = p_rp_list[i+1]
    # We add each approximation
    eal += (loss1 + loss2)*(rp1-rp2)/2
# This is the final trapezoid to add in
final_eal = eal + hazus_def[loss_list[-1]]*p_rp_list[-1]

# Add eal column to our dataframe
hazus_def = pd.concat([hazus_def, pd.Series(final_eal, name='eal')], axis=1)

# Write out the default damages df
hazus_def_out_filep = join(FO, 'benchmark_loss.pqt')
prepare_saving(hazus_def_out_filep)
hazus_def.to_parquet(hazus_def_out_filep)

Estimate Losses for Hazus Default, RP: 500
Estimate Losses for Hazus Default, RP: 100
Estimate Losses for Hazus Default, RP: 50
Estimate Losses for Hazus Default, RP: 10
